In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
    !pip install transformers accelerate datasets peft sqlglot
else:
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth


In [ ]:
import json
import re
import sqlite3
import random
import torch
from datasets import Dataset
from unsloth import FastLanguageModel

In [ ]:
with open("train.json") as f:
    train_bird = json.load(f)

with open("dev.json") as f:
    dev_bird = json.load(f)

print("BIRD Train:", len(train_bird))
print("BIRD Dev:", len(dev_bird))


BIRD Train: 9428
BIRD Dev: 1534


In [ ]:
def convert_bird(item):
    return {
        "instruction": item["question"],
        "output": item["SQL"],
    }

train_bird_prompts = [convert_bird(x) for x in train_bird]
dev_bird_prompts = [convert_bird(x) for x in dev_bird]

print("Converted BIRD:", len(train_bird_prompts), len(dev_bird_prompts))


Converted BIRD: 9428 1534


In [ ]:
with open("train_placement.json") as f:
    train_place = json.load(f)

with open("dev_placement.json") as f:
    dev_place = json.load(f)

with open("dev_tied_append_placement.json") as f:
    dev_place_extra = json.load(f)

dev_place = dev_place + dev_place_extra

print("Placement Train:", len(train_place))
print("Placement Dev:", len(dev_place))


Placement Train: 20
Placement Dev: 8


In [ ]:
placement_schema = """
users(id, email, role, is_approved)
students(id, user_id, first_name, last_name, student_id, department, cgpa)
companies(id, user_id, company_name, industry, location)
jobs(id, company_id, title, description, job_type, ctc_offered, created_at)
applications(id, student_id, job_id, status, applied_at)
"""


In [ ]:
def convert_place(item):
    q = item["question"]
    sql = item["SQL"]
    return {
        "instruction": f"Given the following schema:\n{placement_schema}\nTranslate this question into SQL:\n{q}",
        "output": sql,
    }

train_place_prompts = [convert_place(x) for x in train_place]
dev_place_prompts = [convert_place(x) for x in dev_place]

print("Converted Placement:", len(train_place_prompts), len(dev_place_prompts))


Converted Placement: 20 8


In [ ]:
train_all = train_bird_prompts + train_place_prompts
dev_all = dev_bird_prompts + dev_place_prompts

print("Total Training Samples:", len(train_all))
print("Total Dev Samples:", len(dev_all))


Total Training Samples: 9448
Total Dev Samples: 1542


In [ ]:
original_len = len(train_all)
reduced_len = original_len // 10

random.seed(42)
train_all = random.sample(train_all, reduced_len)

print(f"Reduced training set from {original_len} → {len(train_all)} samples")

Reduced training set from 9448 → 944 samples


In [ ]:
model_name = "mistralai/Mistral-7B-Instruct-v0.2"
max_seq_length = 1024   # reduces training time

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    load_in_4bit=True,
    dtype=torch.float16,
)

FastLanguageModel.for_training(model)

model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
)


==((====))==  Unsloth 2025.11.2: Fast Mistral patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
def tokenize(batch):
    texts = [
        f"### Instruction:\n{instr}\n\n### Response:\n{out}"
        for instr, out in zip(batch["instruction"], batch["output"])
    ]

    enc = tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=max_seq_length,
        return_tensors=None,
    )

    enc["labels"] = enc["input_ids"].copy()
    return enc

train_tokenized = train_dataset.map(
    tokenize,
    batched=True,
    remove_columns=train_dataset.column_names,
)

dev_tokenized = dev_dataset.map(
    tokenize,
    batched=True,
    remove_columns=dev_dataset.column_names,
)


Map:   0%|          | 0/9448 [00:00<?, ? examples/s]

Map:   0%|          | 0/1542 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForLanguageModeling
import os

os.environ["WANDB_DISABLED"] = "true"

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

args = TrainingArguments(
    output_dir="mistral-sql",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,

    max_steps=200,      # TRAIN ONLY 200 STEPS
    warmup_steps=20,

    fp16=True,
    bf16=False,

    logging_steps=20,
    eval_strategy="no",
    save_strategy="no",
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_tokenized,
    data_collator=data_collator,
)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
trainer.train()

model.save_pretrained("fine-tuned-mistral")
tokenizer.save_pretrained("fine-tuned-mistral")


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 9,448 | Num Epochs = 1 | Total steps = 200
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 7,283,675,136 (0.58% trained)


Step,Training Loss
20,0.000200
40,0.005700
60,0.001400
80,0.000500
100,0.000300
120,0.000300
140,0.000300
160,0.000300
180,0.000300
200,0.000300


('fine-tuned-mistral/tokenizer_config.json',
 'fine-tuned-mistral/special_tokens_map.json',
 'fine-tuned-mistral/chat_template.jinja',
 'fine-tuned-mistral/tokenizer.model',
 'fine-tuned-mistral/added_tokens.json',
 'fine-tuned-mistral/tokenizer.json')

In [ ]:
def normalize(sql):
    sql = sql.strip().lower()
    sql = re.sub(r"\s+", " ", sql)
    return sql.rstrip(";")

def exact_match(pred, gold):
    return normalize(pred) == normalize(gold)

def execution_accuracy(conn, pred, gold):
    try:
        p = conn.execute(pred).fetchall()
        g = conn.execute(gold).fetchall()
        return p == g
    except:
        return False


In [ ]:
EM = 0
total = len(dev_place_prompts)

for sample in dev_place_prompts:
    gold = sample["output"]
    prompt = sample["instruction"]

    enc = tokenizer(prompt, return_tensors="pt").to(model.device)
    out = model.generate(**enc, max_new_tokens=150, do_sample=False)

    pred = tokenizer.decode(out[0], skip_special_tokens=True)
    pred = pred.split("### Response:")[-1].strip()

    if exact_match(pred, gold):
        EM += 1

print("Exact Match Accuracy:", EM / total)


Exact Match Accuracy: 0.0


In [ ]:
def token_overlap(pred, gold):
    p_tokens = set(normalize(pred).split())
    g_tokens = set(normalize(gold).split())
    if len(g_tokens) == 0:
        return 0
    return len(p_tokens & g_tokens) / len(g_tokens)

overlap_scores = []

for sample in dev_place_prompts:
    gold = sample["output"]
    prompt = sample["instruction"]

    enc = tokenizer(prompt, return_tensors="pt").to(model.device)
    out = model.generate(**enc, max_new_tokens=150, do_sample=False)

    pred = tokenizer.decode(out[0], skip_special_tokens=True)
    pred = pred.split("### Response:")[-1].strip()

    overlap_scores.append(token_overlap(pred, gold))

print("Token Overlap Accuracy:", sum(overlap_scores) / len(overlap_scores))


Token Overlap Accuracy: 0.6300347222222222


In [ ]:
import sqlglot

def structure_match(pred, gold):
    try:
        p_ast = sqlglot.parse_one(pred)
        g_ast = sqlglot.parse_one(gold)
        return p_ast == g_ast
    except:
        return False

SM = 0
for sample in dev_place_prompts:
    gold = sample["output"]
    prompt = sample["instruction"]

    enc = tokenizer(prompt, return_tensors="pt").to(model.device)
    out = model.generate(**enc, max_new_tokens=150, do_sample=False)
    pred = tokenizer.decode(out[0], skip_special_tokens=True).split("### Response:")[-1].strip()

    if structure_match(pred, gold):
        SM += 1

print("SQL Structure Accuracy:", SM / len(dev_place_prompts))


SQL Structure Accuracy: 0.0


In [ ]:
import sqlglot

# ----------- Helper Functions -----------

def normalize(sql):
    sql = sql.strip().lower()
    sql = re.sub(r"\s+", " ", sql)
    return sql.rstrip(";")

def exact_match(pred, gold):
    return normalize(pred) == normalize(gold)

def token_overlap(pred, gold):
    p = set(normalize(pred).split())
    g = set(normalize(gold).split())
    if len(g) == 0:
        return 0
    return len(p & g) / len(g)

def structure_match(pred, gold):
    try:
        p_ast = sqlglot.parse_one(pred)
        g_ast = sqlglot.parse_one(gold)
        return p_ast == g_ast
    except:
        return False

# ----------- Evaluation Loop -----------

EM = 0
SM = 0
token_scores = []

total = len(dev_place_prompts)

for sample in dev_place_prompts:
    gold = sample["output"]
    prompt = sample["instruction"]

    enc = tokenizer(prompt, return_tensors="pt").to(model.device)
    out = model.generate(**enc, max_new_tokens=150, do_sample=False)

    pred = tokenizer.decode(out[0], skip_special_tokens=True)
    pred = pred.split("### Response:")[-1].strip()

    # Exact Match
    if exact_match(pred, gold):
        EM += 1

    # SQL Structure Match
    if structure_match(pred, gold):
        SM += 1

    # Token Overlap
    token_scores.append(token_overlap(pred, gold))

# ----------- Print Results -----------

print(f"Exact Match Accuracy:       {EM/total:.4f}")
print(f"Token Overlap Accuracy:     {sum(token_scores)/total:.4f}")
print(f"SQL Structure Accuracy:     {SM/total:.4f}")


Exact Match Accuracy:       0.0000
Token Overlap Accuracy:     0.6300
SQL Structure Accuracy:     0.0000


In [ ]:
# Combined DB-free evaluation cell
import re
import sqlglot
import sqlglot.expressions as exp
from difflib import SequenceMatcher
from collections import Counter

# ---------------- Helpers ----------------
def normalize(sql: str):
    if sql is None:
        return ""
    s = sql.strip()
    # remove trailing semicolon and lower
    s = re.sub(r";\s*$", "", s.strip(), flags=re.MULTILINE)
    s = re.sub(r"\s+", " ", s)
    return s.lower().strip()

def exact_match(pred, gold):
    return normalize(pred) == normalize(gold)

def token_overlap(pred, gold):
    p = set(normalize(pred).split())
    g = set(normalize(gold).split())
    return 0.0 if len(g) == 0 else len(p & g) / len(g)

def jaccard(pred, gold):
    p = set(normalize(pred).split())
    g = set(normalize(gold).split())
    denom = len(p | g)
    return 0.0 if denom == 0 else len(p & g) / denom

def edit_ratio(pred, gold):
    # difflib ratio in [0,1] where 1 = identical
    return SequenceMatcher(None, normalize(pred), normalize(gold)).ratio()

def ast_equal(pred, gold):
    try:
        p = sqlglot.parse_one(pred)
        g = sqlglot.parse_one(gold)
        return p == g
    except Exception:
        return False

def extract_tables(sql):
    try:
        t = sqlglot.parse_one(sql)
        tables = {node.name.lower() for node in t.find_all(exp.Table)}
        return tables
    except Exception:
        # fallback naive extraction from FROM/JOIN tokens
        m = re.findall(r"(?:from|join)\s+([`\"]?[\w\.]+[`\"]?)", sql, flags=re.I)
        return {x.replace('"','').replace("`","").split('.')[-1].lower() for x in m}

def extract_columns(sql):
    try:
        t = sqlglot.parse_one(sql)
        cols = set()
        for c in t.find_all(exp.Column):
            # column may come as table.col; keep both forms
            col_sql = c.sql(dialect=None).lower()
            # normalize quotes and schema qualifiers
            col_sql = col_sql.replace('"','').replace("`","")
            cols.add(col_sql)
        return cols
    except Exception:
        # naive: get words between select and from, split by commas
        try:
            sel = re.search(r"select\s+(.*?)\s+from\s", sql, flags=re.I|re.S)
            if not sel:
                return set()
            parts = [p.strip().lower() for p in sel.group(1).split(",")]
            return {p for p in parts if p}
        except Exception:
            return set()

def extract_where_clause(sql):
    try:
        t = sqlglot.parse_one(sql)
        where = next(t.find_all(exp.Where), None)
        if where is None:
            return ""
        return normalize(where.sql())
    except Exception:
        m = re.search(r"where\s+(.*?)(?:group by|order by|limit|$)", sql, flags=re.I|re.S)
        return normalize(m.group(1)) if m else ""

def clause_match(pred, gold, clause):  # clause in ["select","from","where"]
    try:
        if clause == "select":
            pcols = extract_columns(pred)
            gcols = extract_columns(gold)
            if not gcols:
                return 0.0
            return len(pcols & gcols) / len(gcols)
        if clause == "from":
            pt = extract_tables(pred)
            gt = extract_tables(gold)
            if not gt:
                return 0.0
            return len(pt & gt) / len(gt)
        if clause == "where":
            pw = extract_where_clause(pred)
            gw = extract_where_clause(gold)
            if not gw:
                return 0.0
            # simple token overlap in where clause
            return token_overlap(pw, gw)
    except Exception:
        return 0.0

# keywords to check presence
KEYWORDS = ["select","from","where","join","group by","order by","limit","having"]

def keyword_accuracy(pred, gold):
    pred_norm = normalize(pred)
    gold_norm = normalize(gold)
    # we count keywords that are present in gold and check presence in pred
    present = 0
    total = 0
    for kw in KEYWORDS:
        if kw in gold_norm:
            total += 1
            if kw in pred_norm:
                present += 1
    return 1.0 if total == 0 else present/total

# ---------------- Aggregation ----------------
EM = 0
AST = 0
token_overlaps = []
jaccards = []
edit_scores = []
select_scores = []
from_scores = []
where_scores = []
kw_scores = []
col_match_scores = []
table_match_scores = []

# For inspection
DEBUG_N = 6
debug_list = []

# Iterate and evaluate
for i, sample in enumerate(dev_place_prompts):
    gold = sample["output"]
    prompt = sample["instruction"]

    # generate
    enc = tokenizer(prompt, return_tensors="pt").to(model.device)
    out = model.generate(**enc, max_new_tokens=150, do_sample=False)
    pred_raw = tokenizer.decode(out[0], skip_special_tokens=True)
    pred = pred_raw.split("### Response:")[-1].strip()

    # metrics
    if exact_match(pred, gold):
        EM += 1
    if ast_equal(pred, gold):
        AST += 1

    token_overlaps.append(token_overlap(pred, gold))
    jaccards.append(jaccard(pred, gold))
    edit_scores.append(edit_ratio(pred, gold))

    s_match = clause_match(pred, gold, "select")
    f_match = clause_match(pred, gold, "from")
    w_match = clause_match(pred, gold, "where")
    select_scores.append(s_match)
    from_scores.append(f_match)
    where_scores.append(w_match)

    kw_scores.append(keyword_accuracy(pred, gold))

    # columns/tables matching
    gcols = extract_columns(gold)
    pcols = extract_columns(pred)
    col_match_scores.append(0.0 if not gcols else len(pcols & gcols) / len(gcols))

    gtables = extract_tables(gold)
    ptables = extract_tables(pred)
    table_match_scores.append(0.0 if not gtables else len(ptables & gtables) / len(gtables))

    # debug capture
    if i < DEBUG_N:
        debug_list.append({
            "idx": i,
            "prompt": prompt,
            "gold": gold,
            "pred": pred,
            "select_match": s_match,
            "from_match": f_match,
            "where_match": w_match,
            "token_overlap": token_overlaps[-1],
            "jaccard": jaccards[-1],
            "edit_ratio": edit_scores[-1],
            "ast_equal": ast_equal(pred, gold)
        })

# ------------- Print Summary -------------
n = len(dev_place_prompts)
print("----- Aggregate Scores -----")
print(f"Total samples evaluated: {n}")
print(f"Exact Match (EM): {EM}/{n} = {EM/n:.4f}")
print(f"AST Structure Match: {AST}/{n} = {AST/n:.4f}")
print(f"Mean Token Overlap: {sum(token_overlaps)/n:.4f}")
print(f"Mean Jaccard: {sum(jaccards)/n:.4f}")
print(f"Mean Edit Ratio (difflib): {sum(edit_scores)/n:.4f}")
print()
print("Clause partial matches (mean):")
print(f"  SELECT: {sum(select_scores)/n:.4f}")
print(f"  FROM:   {sum(from_scores)/n:.4f}")
print(f"  WHERE:  {sum(where_scores)/n:.4f}")
print()
print(f"Mean Keyword Accuracy: {sum(kw_scores)/n:.4f}")
print(f"Mean Column Match: {sum(col_match_scores)/n:.4f}")
print(f"Mean Table Match: {sum(table_match_scores)/n:.4f}")

# ------------- Print debug examples -------------
print("\n----- Example outputs (first few) -----")
for d in debug_list:
    print(f"\nSample #{d['idx']+1}")
    print("PROMPT:", d["prompt"][:180].replace("\n"," ") + "...")
    print("GOLD :", d["gold"])
    print("PRED :", d["pred"])
    print(f"SELECT match: {d['select_match']:.3f} | FROM match: {d['from_match']:.3f} | WHERE match: {d['where_match']:.3f}")
    print(f"Token overlap: {d['token_overlap']:.3f} | Jaccard: {d['jaccard']:.3f} | Edit ratio: {d['edit_ratio']:.3f}")
    print(f"AST equal: {d['ast_equal']}")


----- Aggregate Scores -----
Total samples evaluated: 8
Exact Match (EM): 0/8 = 0.0000
AST Structure Match: 0/8 = 0.0000
Mean Token Overlap: 0.6595
Mean Jaccard: 0.1271
Mean Edit Ratio (difflib): 0.1630

Clause partial matches (mean):
  SELECT: 0.0455
  FROM:   0.8750
  WHERE:  0.4643

Mean Keyword Accuracy: 0.9688
Mean Column Match: 0.0455
Mean Table Match: 0.8750

----- Example outputs (first few) -----

Sample #1
PROMPT: Given the following schema:  users(id, email, role, is_approved) students(id, user_id, first_name, last_name, student_id, department, cgpa) companies(id, user_id, company_name, ind...
GOLD : SELECT DISTINCT c.company_name FROM companies c JOIN jobs j ON c.id = j.company_id WHERE MONTH(j.created_at) = 10
PRED : Given the following schema:

users(id, email, role, is_approved)
students(id, user_id, first_name, last_name, student_id, department, cgpa)
companies(id, user_id, company_name, industry, location)
jobs(id, company_id, title, description, job_type, ctc_offered,

In [ ]:
EM = 0
EX = 0
total = len(dev_place_prompts)

conn = sqlite3.connect("placement.sqlite")  # your DB with placement schema

for sample in dev_place_prompts:
    gold = sample["output"]
    prompt = sample["instruction"]

    enc = tokenizer(prompt, return_tensors="pt").to(model.device)
    out = model.generate(**enc, max_new_tokens=150, do_sample=False)

    pred = tokenizer.decode(out[0], skip_special_tokens=True)
    pred = pred.split("### Response:")[-1].strip()

    if exact_match(pred, gold):
        EM += 1

    if execution_accuracy(conn, pred, gold):
        EX += 1

print("Exact Match Accuracy:", EM / total)
print("Execution Accuracy:", EX / total)
print("Final Evaluation Accuracy:", (EM/total + EX/total) / 2)


Exact Match Accuracy: 0.0
Execution Accuracy: 0.0
Final Evaluation Accuracy: 0.0


In [ ]:
def ask(query, schema):
    prompt = f"### Instruction:\nGiven the schema:\n{schema}\nTranslate this question into SQL:\n{query}\n\n### Response:\n"
    enc = tokenizer(prompt, return_tensors="pt").to(model.device)
    out = model.generate(**enc, max_new_tokens=150, do_sample=False)
    return tokenizer.decode(out[0], skip_special_tokens=True).split("### Response:")[-1].strip()

ask("List all students with CGPA above 8", placement_schema)


'```sql\nSELECT s.id, s.user_id, s.first_name, s.last_name, s.student_id, s.department, s.cgpa\nFROM students s\nWHERE s.cgpa > 8;\n```'

In [ ]:
!zip -r fine-tuned-mistral.zip fine-tuned-mistral

  adding: fine-tuned-mistral/ (stored 0%)
  adding: fine-tuned-mistral/README.md (deflated 65%)
  adding: fine-tuned-mistral/tokenizer.json (deflated 85%)
  adding: fine-tuned-mistral/tokenizer.model (deflated 55%)
  adding: fine-tuned-mistral/adapter_model.safetensors (deflated 7%)
  adding: fine-tuned-mistral/tokenizer_config.json (deflated 68%)
  adding: fine-tuned-mistral/special_tokens_map.json (deflated 79%)
  adding: fine-tuned-mistral/adapter_config.json (deflated 57%)
  adding: fine-tuned-mistral/chat_template.jinja (deflated 64%)


In [ ]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 91.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 139.8 MB/s eta 0:00:00


In [ ]:
!ngrok config add-authtoken 33CPxN7YfOrNiC6gDQWv2rM7LEi_2Q36iGpiXACw5m73poxTx

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok

# Terminate any existing tunnels
ngrok.kill()

# Start a new tunnel
public_url = ngrok.connect(8501, "http")
print(f"Streamlit app is live at: {public_url}")

# Run the Streamlit app
!streamlit run /content/app.py

Streamlit app is live at: NgrokTunnel: "https://2e3d7bcda07f.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.168.212.77:8501



🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
2025-09-25 16:40:44.880170: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758818444.917821   22952 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758818444.929311   22952 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1758818444.957946   22952 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1758818444.957985   22952 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:17

In [ ]:
!pip install pyngrok


In [ ]:
!zip -r fine-tuned-mistral.zip fine-tuned-mistral

  adding: fine-tuned-mistral/ (stored 0%)
  adding: fine-tuned-mistral/adapter_model.safetensors (deflated 8%)
  adding: fine-tuned-mistral/README.md (deflated 65%)
  adding: fine-tuned-mistral/tokenizer.model (deflated 55%)
  adding: fine-tuned-mistral/adapter_config.json (deflated 58%)
  adding: fine-tuned-mistral/tokenizer_config.json (deflated 68%)
  adding: fine-tuned-mistral/special_tokens_map.json (deflated 79%)
  adding: fine-tuned-mistral/tokenizer.json (deflated 85%)


In [ ]:
from google.colab import files
files.download("fine-tuned-mistral.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>